In [9]:
#all packages which will be used
import numpy as np
import pandas as pd
from scipy.io import loadmat
import time
import random

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans

#plot
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from matplotlib.axes import Axes

In [3]:
X_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_resampling3000_X.csv")
y_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_resampling3000_y.csv")

In [4]:
X_train=X_train.drop(['robust_dis'],axis=1)
y_train=y_train.drop(['robust_dis'],axis=1)
y_train=y_train.rename(columns={'0':'label'})

In [14]:
#boundary detection
class boundary_detection(BaseEstimator,TransformerMixin):
    def __init__(self,labels,kmeans):
        self.labels=labels
        self.kmeans=kmeans
    def fit(self,X,y=None):
        start=time.time()
        print("fit start")
        self.kmeans.fit(X)
        print("fit end")
        print("fit time:%5.1fminute"%((time.time()-start)/60))
        return self
    def transform(self,X,y=None):
        #dataset=pd.concat([X,self.labels],axis=1)
        count1=X.shape[0]
        predict_cluster = self.kmeans.predict(X)
        predict_cluster=pd.DataFrame( predict_cluster,columns=['predict_cluster'])
        X=pd.concat([X,predict_cluster],axis=1)
        self.labels=pd.concat([self.labels,predict_cluster],axis=1)
        #center_set = pd.DataFrame(columns = ['feature0','feature1','feature2','feature3','feature4','predict_label'])
        center_set = list()
        i = 0
        #print (X)
        #print (self.kmeans.cluster_centers_)
        print("drop start")
        for cluster in range(np.int((X.shape[0])/100)):
            if cluster%1000==0:
                print("cluster number:",cluster)
            temp_list = list(self.labels[self.labels['predict_cluster']==cluster]['label'])
            if 1 and 2 in temp_list:
                pass
            elif temp_list == []:  
                pass
            else:
                #temp =temp_list[0]
                #temp_center = np.append(self.kmeans.cluster_centers_[cluster],temp)
                temp_center_set = {'feature0':self.kmeans.cluster_centers_[cluster][0],\
                                  'feature1':self.kmeans.cluster_centers_[cluster][1],\
                                  'feature2':self.kmeans.cluster_centers_[cluster][2],\
                                  'feature3':self.kmeans.cluster_centers_[cluster][3],\
                                  'feature4':self.kmeans.cluster_centers_[cluster][4],\
                                  'predict_label':temp_list[0]}
                #temp = pd.DataFrame(data=temp_center_set,index = cluster)
                #center_set = pd.concat(center_set,temp_center_set)
                center_set.append(temp_center_set)
                self.labels = self.labels[self.labels.predict_cluster !=cluster]
                X = X[X.predict_cluster != cluster]
        #dataset.append(pd.DataFrame(self.kmeans.cluster_centers_))#save all the centers of abandoned clusters
        print("drop end")
        count2=X.shape[0]
        print("Drop count:",count1-count2)        
        return X,self.labels,center_set

In [15]:
#apply the function with 30 samples in one cluster
X_boundary,y_boundary,center=boundary_detection(labels=y_train.drop('Unnamed: 0',1),kmeans=KMeans(n_clusters=np.int((X_train.shape[0])/100))).fit_transform(X_train)

fit start


MemoryError: 

In [ ]:
len(y_boundary[y_boundary['label']==1])

In [ ]:
len(y_boundary[y_boundary['label']==2])

In [ ]:
center_set = pd.DataFrame(data = center)

In [ ]:
train = {'feature0':X_boundary['feature0'],'feature1':X_boundary['feature1'],'feature2':X_boundary['feature2'],'feature3':X_boundary['feature3'],'feature4':X_boundary['feature4'],'predict_label':y_boundary['label']}
train = pd.DataFrame(data=train)
train_combination = [train, center_set]
train_combination = pd.concat(train_combination)
train_combination

In [ ]:
import re
dataset = loadmat ("D:\\lab; signal processing\\forStudents\\medData\\dataset.mat",mat_dtype=True)
data = dataset ["dataset"]
cmap1 = matplotlib.colors.LinearSegmentedColormap.from_list('my_cmap',['black','green','blue'],256)
def check_resultofdropping (data,stuck,index,slices,features,selected):
    target_data=data[stuck][0][0][0][0][:,:,slices,features]
    target_image1=target_data.reshape(target_data.shape[0],target_data.shape[1])
    target_label=data[stuck][0][0][0][index][:,:,slices]
    target_image2=target_label.reshape(target_label.shape[0],target_label.shape[1])
    plt.imshow(target_image1,cmap = matplotlib.cm.binary,alpha=0.4)
    plt.imshow(target_image2,cmap = cmap1,interpolation="bilinear",alpha=0.4)
    plt.axis("off") #close the axis number

    selected_x=[]
    selected_y=[]
    selected_matrix = selected[selected['sourceofpixel']==stuck]
    selected_matrix=pd.DataFrame.reset_index(selected_matrix).drop('index',axis=1)
    for i in range(len(selected_matrix)):
        temparr = re.findall("\d+",selected_matrix.iloc[i]['indexofpixel']) 
        if (int(temparr[2]) == slices):
            selected_x.append(int(temparr[0]))
            selected_y.append(int(temparr[1]))
    plt.scatter(selected_y,selected_x, s = 3)
    plt.show()
    
    target_data=data[stuck][0][0][0][0][:,:,slices,features]
    target_image1=target_data.reshape(target_data.shape[0],target_data.shape[1])
    target_label=data[stuck][0][0][0][index][:,:,slices]
    target_image2=target_label.reshape(target_label.shape[0],target_label.shape[1])
    plt.imshow(target_image1,cmap = matplotlib.cm.binary,alpha=0.4)
    plt.imshow(target_image2,cmap = cmap1,interpolation="bilinear",alpha=0.4)
    plt.axis("off") #close the axis number
    plt.show()

In [ ]:
stuck,slices,index,features=4,24,1,4
check_resultofdropping(data,stuck,index,slices,features,y_boundary)

In [ ]:
#save the result
X_boundary.to_csv("D:\\lab; signal processing\\forStudents\\medData\\train_x_resample3000_boundarysel_50pergroup_kmeans.csv",index=False)
y_boundary.to_csv("D:\\lab; signal processing\\forStudents\\medData\\train_y_resample3000_boundarysel_50pergroup_kmeans.csv",index=False)
train_combination.to_csv("D:\\lab; signal processing\\forStudents\\medData\\train_combination_resample3000_boundarysel_50pergroup_kmeans.csv",index=False)